# Auto_Ml on a subset of the datacube

In [2]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import xarray as xr
import rioxarray as rxr
import geopandas as gpd
import harmonize as hz

In [26]:
# Open the dataset with xarray
path_data = "../data/final/"

In [27]:
# Open the datacube5 as an xarray dataset
datacube = xr.open_dataset(path_data+ 'datacube.nc')

In [28]:
datacube

<xarray.Dataset>
Dimensions:            (x: 248, y: 217, time: 4019)
Coordinates:
  * x                  (x) float64 3.294e+05 3.303e+05 ... 5.574e+05 5.583e+05
  * y                  (y) float64 5.032e+06 5.031e+06 ... 4.833e+06 4.832e+06
  * time               (time) object 2010-01-01 00:00:00 ... 2021-01-01 00:00:00
    band               int32 ...
Data variables: (12/16)
    crs                int32 ...
    ET_500m            (time, y, x) float32 ...
    Fpar_500m          (time, y, x) float64 ...
    spatial_ref        int32 ...
    density            (y, x) float32 ...
    u10                (time, y, x) float64 ...
    ...                 ...
    Last_Day           (time, y, x) float64 ...
    Burn_Date          (time, y, x) float64 ...
    FireMask           (time, y, x) float32 ...
    LST_Day_1km        (time, y, x) float64 ...
    LST_Night_1km      (time, y, x) float64 ...
    _1_km_16_days_EVI  (time, y, x) float64 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-09-15 13:57:35 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [6]:
D# Cut the original datacube to a smaller one: half of the full time
datacube = datacube.isel(time=slice(180, 250))


NameError: name 'D' is not defined

We have a data set with 16 columns and datarray 19,696,656 rows. Let's look at the columns.


In [29]:
df = datacube.to_dataframe()

MemoryError: Unable to allocate 1.61 GiB for an array with shape (216286504,) and data type int64

In [9]:
df.head(20)

crs  ET_500m  Fpar_500m  band  \
x             y            time                                                 
329415.341451 5.032039e+06 2010-01-01 00:00:00    0      NaN        NaN     1   
                           2010-01-02 00:00:00    0      NaN        NaN     1   
                           2010-01-03 00:00:00    0      NaN        NaN     1   
                           2010-01-04 00:00:00    0      NaN        NaN     1   
                           2010-01-05 00:00:00    0      NaN        NaN     1   
                           2010-01-06 00:00:00    0      NaN        NaN     1   
                           2010-01-07 00:00:00    0      NaN        NaN     1   
                           2010-01-08 00:00:00    0      NaN        NaN     1   
                           2010-01-09 00:00:00    0      NaN        NaN     1   
                           2010-01-10 00:00:00    0      NaN        NaN     1   
                           2010-01-11 00:00:00    0      NaN        NaN     1   
                           2010-01-12 00:00:00    0      NaN        NaN     1   
                           2010-01-13 00:00:00    0      NaN        NaN     1   
                           2010-01-14 00:00:00    0      NaN        NaN     1   
                           2010-01-15 00:00:00    0      NaN        NaN     1   
                           2010-01-16 00:00:00    0      NaN        NaN     1   
                           2010-01-17 00:00:00    0      NaN        NaN     1   
                           2010-01-18 00:00:00    0      NaN        NaN     1   
                           2010-01-19 00:00:00    0      NaN        NaN     1   
                           2010-01-20 00:00:00    0      NaN        NaN     1   

                                                spatial_ref  density  u10  \
x             y            time                                             
329415.341451 5.032039e+06 2010-01-01 00:00:00            0      NaN  NaN   
                           2010-01-02 00:00:00            0      NaN  NaN   
                           2010-01-03 00:00:00            0      NaN  NaN   
                           2010-01-04 00:00:00            0      NaN  NaN   
                           2010-01-05 00:00:00            0      NaN  NaN   
                           2010-01-06 00:00:00            0      NaN  NaN   
                           2010-01-07 00:00:00            0      NaN  NaN   
                           2010-01-08 00:00:00            0      NaN  NaN   
                           2010-01-09 00:00:00            0      NaN  NaN   
                           2010-01-10 00:00:00            0      NaN  NaN   
                           2010-01-11 00:00:00            0      NaN  NaN   
                           2010-01-12 00:00:00            0      NaN  NaN   
                           2010-01-13 00:00:00            0      NaN  NaN   
                           2010-01-14 00:00:00            0      NaN  NaN   
                           2010-01-15 00:00:00            0      NaN  NaN   
                           2010-01-16 00:00:00            0      NaN  NaN   
                           2010-01-17 00:00:00            0      NaN  NaN   
                           2010-01-18 00:00:00            0      NaN  NaN   
                           2010-01-19 00:00:00            0      NaN  NaN   
                           2010-01-20 00:00:00            0      NaN  NaN   

                                                v10  t2m  tp  First_Day  \
x             y            time                                           
329415.341451 5.032039e+06 2010-01-01 00:00:00  NaN  NaN NaN        NaN   
                           2010-01-02 00:00:00  NaN  NaN NaN        NaN   
                           2010-01-03 00:00:00  NaN  NaN NaN        NaN   
                           2010-01-04 00:00:00  NaN  NaN NaN        NaN   
                           2010-01-05 00:00:00  NaN  NaN NaN        NaN   
                           2010-01-06 00:00:0

In [10]:
# Drop the columns we don't need: crs, band, spatial_ref
df = df.drop(columns=['crs', 'band', 'spatial_ref'])

In [11]:
# Count the percentage of missing values in each column
df.isnull().sum() / len(df)

ET_500m              0.072618
Fpar_500m            0.007860
density              0.069942
u10                  0.203498
v10                  0.203498
t2m                  0.203498
tp                   0.203498
First_Day            0.012365
Last_Day             0.012365
Burn_Date            0.012365
FireMask             0.002081
LST_Day_1km          0.003486
LST_Night_1km        0.002898
_1_km_16_days_EVI    0.002081
dtype: float64

In [12]:
# Drop all the missing values
df = df.dropna()

In [13]:
# Check the percentage of missing values in each column
df.isnull().sum() / len(df)

ET_500m              0.0
Fpar_500m            0.0
density              0.0
u10                  0.0
v10                  0.0
t2m                  0.0
tp                   0.0
First_Day            0.0
Last_Day             0.0
Burn_Date            0.0
FireMask             0.0
LST_Day_1km          0.0
LST_Night_1km        0.0
_1_km_16_days_EVI    0.0
dtype: float64

In [14]:
# Structure of the dataframe
df.shape

(15570006, 14)

In [15]:
# Count the number of unique values in FireMask
df['FireMask'].value_counts()

5.0    14993489
4.0      534871
3.0       40726
8.0         848
9.0          48
7.0          24
Name: FireMask, dtype: int64

In [16]:
# Change the value of FireMAsk to 0 and 1. 0 = no fire, 1 = fire . If FireMask is 7,8,9 then it is a fire. If not, it is not a fire.
df['FireMask'] = df['FireMask'].replace([0,1,2,3,4,5,6], 0)
df['FireMask'] = df['FireMask'].replace([7,8,9], 1)


In [17]:
# Counthe number of fires and no fires
df['FireMask'].value_counts()


0.0    15569086
1.0         920
Name: FireMask, dtype: int64

In [19]:
# Keep all the observations with FireMask = 1 and keep 1000 observations with FireMask = 0 randomly
df = df[df['FireMask'] == 1].append(df[df['FireMask'] == 0].sample(n=1000, random_state=1))


C:\Users\Quentin\AppData\Local\Temp\ipykernel_12056\2366296675.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df[df['FireMask'] == 1].append(df[df['FireMask'] == 0].sample(n=1000, random_state=1))


In [20]:
df['FireMask'].value_counts()

0.0    1000
1.0     920
Name: FireMask, dtype: int64

In [22]:
# export the dataframe to a csv file
df.to_csv(path_data + 'datacube_balanced.csv')

In [25]:
from pycaret.classification import *

ModuleNotFoundError: No module named 'pycaret'